### Input
Definimos a primieira camada e passamos o tamanho da nossa imagem, para sabermos a quantidade de inputs que teremos na nossa rede neural

imagem 32x32 cinza:

```python
layers.Input(32,32,1)
```

imagem 32x32 cinza:

```python
layers.Input(32,32,3)
```


---

### Flatten
Utilizado para achatar a nossa imagem, portanto se temos uma imagem cinza de 32x32 teremos então um array de 1024 (32*32). 


E para uma imagem 32x32 porém RGB será um array de 3072 (32\*32\*3) 

```python
layers.Flatten()
```


---

## 

### Dense
Uma camada de nerônios completamente conectados aos neurônios da camada anterior

```python
# DEFAULT:
layers.Dense(
    units,                                  # Quantidade de neurônios na camada
    activation,                             # Função de ativação 
    use_bias=True,                          # Incluir viés/bias (opcional)
    kernel_initializer="glorot_uniform",    # Tipo do inicializador dos pesos (opcional)
    bias_initializer="zeros",               # Tipo do inicializador dos bias (opcional)
    name=None                               # Nome da camada (opcional)
)

# USE:
layers.Dense(128, activation='relu')
```

![](./img/dense_layer.png)

---

### Padding
Podemos definir o 'padding' que é o comportamento do filtro dentro da imagem, mas especificamente o comportamento com diferentes tamanhos e como vai lidar com as bordas

##### Same
adiciona zeros nas bordas somente se necessário para manter o tamanho da saída o mais próximo possível da entrada.
Quando stride = 1, o tamanho da saída fica idêntico ao da entrada, e o kernel consegue analisar cada pixel (inclusive bordas, com padding de zeros).

##### Valid
aplica-se em todo local onde a pool cabe na imagem, ou seja, as bordas nunca são centralizadas por não adicionamos zeros. Evita valores artificiais (zeros) e pega somente informação real da imagem. Usado quando você aceita perder bordas pra ter features mais limpas.



---

### Conv2D
Convolução 2D (altura e largura)

Inicializa filtros com valores aleatórios que durante o treinamento serão ajustados para identificar coisas úteis para o caso

```python
# DEFAULT:
layers.Conv2D(
    filters,                                        # Quantidade de filtros
    kernel_size=(height,width),                     # Dimensões dos filtros
    strides=(stride_height = 1, stride_width = 1),  # Deslocamento, de quantos em quantos pixels o filtro vai ser aplicado
    padding="valid",                                # Tipo de ação para as bordas
)

# USE:
layers.Conv2D(16, (3,3))
```

---

### MaxPooling
Delimita uma área e pega o maior valor dessa área. 

```python
# DEFAULT:
layers.MaxPooling2D(
    pool_size=(2, 2),   # Dimensão da pool
    strides=None,       # Deslocamento, de quantos em quantos pixels a pool vai ser aplicada
    padding='valid',    # Tipo da ação para as bordas
) 

# USE:
layer.MaxPooling2D(pool_size=(3,3), strides=1, padding='same)

```
![](./img/max_pool.png)



![](./img/max_pool_padding.png)

---

## Aplicação

### Definindo o modelo

#### As imagens no nosso dataset tem dimensões de 28x28 e estão em tons de cinza, portanto temos apenas 1 canal, ou seja, shape=(28,28,1)
```python
layers.Input(shape=(28,28,1))
```
    Output: (28, 28, 1)


### Aplicamos uma camada de filtros com N neuronios para filtros que recebe a saída da camada anterior (28,28,1) resulta em N imagens de saída
```python
layers.Conv2D(10, kernel_size=(3,3), strides=(1,1), padding='same')
```
    Input:  (28, 28, 1)
    Output: (28, 28, 10)


#### Uma camada para transformar a matriz ou cubo de elementos em um unico array
```python
layers.Flatten()
```
    Input:  (28, 28, 10)
    Output: (7840,)


#### Camada com N neuronios, com ativação de relu, ou seja, para valores negativos a saída é 0. Inicilização aleatória próximo de zero para todos os neurônios
```python
layers.Dense(64, activations=activations.relu, kernel_initializer=initializer.RandomNormal())
```
    Input:  (7840,) 
    Output: 64


#### Camada com N neuronios, onde N é a quantidade de saídas possíveis. Com ativação de softmax, ou seja, produz N saídas onde a soma de todas elas resulta em 1.00. Inicilização aleatória próximo de zero para todos os neurônios
```python
layers.Dense(26, activations=activations.softmax, kernel_initializer=initializer.RandomNormal())
```
    Input: 64
    Output: 26


In [2]:
from tensorflow.keras import models, layers, activations, initializers

model = models.Sequential([
    layers.Input(shape=(28,28,3)), # 28x28x1
    layers.Conv2D(10, kernel_size=(3,3), strides=(1,1), padding='same'), # 10 @ 28x28
    layers.Flatten(), # 7840
    layers.Dense(64, activation=activations.relu, kernel_initializer=initializers.RandomNormal()), # 64
    layers.Dense(24, activation=activations.softmax, kernel_initializer=initializers.RandomNormal()) # 24
])

---

### Compilando

##### Optimizer
Quando o modelo realiza uma previsão calculamos o Erro, o optimizer é como vamos ajustar os pesos com base no Erro calculado

- SGD (Stochastic Gradient Descent)	Simples e direto, ajusta pesos gradualmente. Pode ser lento.
- Adam	Mistura de momentum + adaptativo. É o mais usado, aprende mais rápido e estável.
- RMSprop	Parecido com Adam, usado muito em redes recorrentes.
- Adagrad, Adadelta	Adaptam a taxa de aprendizado por parâmetro, mas menos usados hoje.



Momentum: Reduz as oscilações do resultado da melhora do erro. Guarda o histórico dos valores resultantes e utiliza a "velocidade" e a direção que está indo para calcular o novo.

Apatativo: Utiliza sua própria taxa de aprendizado. Se um peso sempre erra muito, o otimizador reduz a taxa pra não oscilar demais. Se um peso erra pouco o otimizador aumenta a taxa pra aprender mais rápido.

Rede recorrente: (Recurrent Neural Netword, RNN) Uma rede feita pra funcionar com dados em sequência. Ex: "O monstro amarelo tem um olho". Treina percorrendo cada palavra e considerando as frase montada antes da palavra do indice atual


![](./img/optimizers.png)


##### Loss
Define como vai ser calculado o erro com a predição. Vai depender da situação analisada:

| Tipo de problema                     | Função de perda mais usada            | String / Classe                        |
|--------------------------------------|---------------------------------------|----------------------------------------|
| Classificação binária                | Binary Crossentropy                   | `'binary_crossentropy'`                |
| Classificação multi-classe (one-hot) | Categorical Crossentropy              | `'categorical_crossentropy'`           |
| Classificação multi-classe (inteiro) | Sparse Categorical Crossentropy       | `'sparse_categorical_crossentropy'`    |
| Regressão                            | Mean Squared Error (MSE)              | `'mse'`, `'mean_squared_error'`        |


##### Metrics
Não altera o funcionamento em si mas podemos definir o que queremos acompanhar enquanto o modelo estiver treinando
| Métrica | Classe / String | Descrição |
|----------|-----------------|------------|
| `Accuracy` | `'accuracy'` / `metrics.Accuracy()` | Calcula a proporção de predições corretas em relação ao total. |
| `BinaryAccuracy` | `'binary_accuracy'` / `metrics.BinaryAccuracy()` | Mede a acurácia em tarefas de classificação binária. |
| `CategoricalAccuracy` | `'categorical_accuracy'` / `metrics.CategoricalAccuracy()` | Mede a acurácia considerando rótulos one-hot. |
| `SparseCategoricalAccuracy` | `'sparse_categorical_accuracy'` / `metrics.SparseCategoricalAccuracy()` | Mede a acurácia considerando rótulos inteiros (não one-hot). |
| `Precision` | `'Precision'` / `metrics.Precision()` | Proporção de predições positivas que realmente são positivas. |
| `Recall` | `'Recall'` / `metrics.Recall()` | Proporção de instâncias positivas que foram corretamente identificadas. |
| `AUC` | `'AUC'` / `metrics.AUC()` | Calcula a área sob a curva ROC (Receiver Operating Characteristic). |
| `TopKCategoricalAccuracy` | `'top_k_categorical_accuracy'` / `metrics.TopKCategoricalAccuracy(k=5)` | Considera correta a predição se o rótulo verdadeiro estiver entre as *k* classes mais prováveis. |
| `F1Score` | `metrics.F1Score()` *(TensorFlow ≥ 2.11)* | Combina *Precision* e *Recall* na fórmula F1 = 2·(P·R)/(P+R). |

In [3]:
from tensorflow.keras import optimizers, losses, metrics

lr = 0.001

model.compile(
    optimizer = optimizers.Adam(
        learning_rate = lr    
    ),
    loss = losses.SparseCategoricalCrossentropy(),
    metrics = [ metrics.sparse_categorical_accuracy ]
)

---

# Lendo o dataset

```python
utils.image_dataset_from_directory(
    directory           
    image_size
    shuffle
    seed
    batch_size
    subset
    validation
    label_mode
)
```

#### directory
    Diretório onde se encontra nosso dataset

#### image_size
    Tamanho o qual as imagens serão lidas. Caso não tenham o tamanho infomrado elas serão redimensionadas.
    Sendo Z e Y os valores definidos. No processamento das imagens vai ser convertido N x M -> Z x Y


#### shuffle
    Mistura as imagens do dataset mas sem perder a coerência

#### seed
    Garante que se rodar o código de novo, o embaralhamento seja igual.

#### batch_size
    Onde definimos o tamanho dos pacotes de dados/imagens que serão usados na hora do treinamento

#### validation & subset
    validation
    Define o tamanho da parte que será dividida como validação do dataset

    subset 
    Define qual parte da divisão este é ('validation' | 'training')

```python
train = utils.image_dataset_from_directory(
    ...,
    validation= 0.2,
    subset= 'training',
    ...
)

test = utils.image_dataset_from_directory(
    ...,
    validation= 0.2,
    subset= 'validation',
    ...
)
```
Como nosso validation é definido com 20% dos nossos dados, o subset validation tem 20% dos dados e o subset training todo o resto

#### label_mode
Define o formato dos rótulos retornados junto com as imagens


In [4]:
from tensorflow.keras import utils

path = './Data'
batch_size = 64

train = utils.image_dataset_from_directory(
    directory=path + '/Train',
    shuffle = True,
    seed = 1,
    image_size = (28,28),
    batch_size = batch_size
)

test = utils.image_dataset_from_directory(
    directory=path + '/Test',
    shuffle = True,
    seed = 1,
    image_size = (28,28),
    batch_size = batch_size
)

Found 27455 files belonging to 24 classes.
Found 7172 files belonging to 24 classes.


**No caso a cima não foi necessário utilizar validation e subset pois já temos dois datasets já separados por validação e treino

---

# Treinando o modelo

In [5]:
from tensorflow.keras import callbacks

patience = 5
epochs = 100

model.fit(
    train,
    validation_data = test,
    epochs = epochs,
    verbose = True,
    
    callbacks = [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = True
        )
    ]
)

Epoch 1/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 47s 107ms/step - loss: 38.3543 - sparse_categorical_accuracy: 0.0439 - val_loss: 3.1876 - val_sparse_categorical_accuracy: 0.0229
Epoch 2/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 3.1765 - sparse_categorical_accuracy: 0.0448 - val_loss: 3.1935 - val_sparse_categorical_accuracy: 0.0229
Epoch 3/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 3.1762 - sparse_categorical_accuracy: 0.0465 - val_loss: 3.1972 - val_sparse_categorical_accuracy: 0.0229
Epoch 4/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 3.1762 - sparse_categorical_accuracy: 0.0469 - val_loss: 3.1991 - val_sparse_categorical_accuracy: 0.0229
Epoch 5/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 3.1761 - sparse_categorical_accuracy: 0.0472 - val_loss: 3.2002 - val_sparse_categorical_accuracy: 0.0229
Epoch 6/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 3.1761 - sparse_categorical_accuracy: 0.0470 - val_loss: 3.2007 - val_sparse_categorical_accura

---

# Melhorando o modelo

In [6]:
from tensorflow.keras import models, layers, activations, initializers

model = models.Sequential([
    layers.Input(shape=(28,28,3)), 
    
    layers.Resizing(28,28),
    layers.Rescaling(1./255),
    layers.RandomRotation((-0.2, 0.2)),

    layers.Conv2D(10, kernel_size=(3,3), strides=(1,1), padding='same'),
    layers.MaxPooling2D((2,2)),
    
    layers.Flatten(),

    layers.Dense(64, activation=activations.relu, kernel_initializer=initializers.RandomNormal()),
    layers.Dropout(0.2),
    layers.Dense(64, activation=activations.relu, kernel_initializer=initializers.RandomNormal()),
    layers.Dense(24, activation=activations.softmax, kernel_initializer=initializers.RandomNormal())
])

from tensorflow.keras import optimizers, losses, metrics

lr = 0.001

model.compile(
    optimizer = optimizers.Adam(
        learning_rate = lr    
    ),
    loss = losses.SparseCategoricalCrossentropy(),
    metrics = [ metrics.sparse_categorical_accuracy ]
)

from tensorflow.keras import callbacks

patience = 5
epochs = 100
model_path = "./model.keras"

model.fit(
    train,
    validation_data = test,
    epochs = epochs,
    verbose = True,
    
    callbacks = [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        callbacks .ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        )
    ]
)

Epoch 1/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 2.9896 - sparse_categorical_accuracy: 0.0888 - val_loss: 2.1978 - val_sparse_categorical_accuracy: 0.2627
Epoch 2/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 2.3049 - sparse_categorical_accuracy: 0.2462 - val_loss: 1.8932 - val_sparse_categorical_accuracy: 0.3484
Epoch 3/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 1.9649 - sparse_categorical_accuracy: 0.3442 - val_loss: 1.5733 - val_sparse_categorical_accuracy: 0.4721
Epoch 4/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 1.6667 - sparse_categorical_accuracy: 0.4242 - val_loss: 1.4705 - val_sparse_categorical_accuracy: 0.4817
Epoch 5/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 1.4277 - sparse_categorical_accuracy: 0.5066 - val_loss: 1.2725 - val_sparse_categorical_accuracy: 0.5555
Epoch 6/100
429/429 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 1.2356 - sparse_categorical_accuracy: 0.5727 - val_loss: 1.1371 - val_sparse_categorical_accuracy: